# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [0]:
# import libraries
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
import pickle

In [3]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
# load data from database
engine = create_engine('sqlite:////content/disaster_dataset.db')
df = pd.read_sql_table("disaster_dataset", engine) 
#df.to_sql('table', myeng, index=False)

In [5]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [0]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [8]:
#test tokenize function
test = 'Hello I see fire in the street and many houses are destroyed, homeless people everywhere'
tokenize(test)

['hello',
 'i',
 'see',
 'fire',
 'in',
 'the',
 'street',
 'and',
 'many',
 'house',
 'are',
 'destroyed',
 ',',
 'homeless',
 'people',
 'everywhere']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [0]:
model = Pipeline([
       ('vect', CountVectorizer(tokenizer=tokenize)),
       ('tfidf', TfidfTransformer()),
       ('clf', MultiOutputClassifier(RandomForestClassifier())),
   ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 42)

#model = []
#model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [0]:
#create a function to easily assess a model's performance.
results_dict = {}

def perf_report(model, X_test, y_test):
    '''
    Function to generate classification report on the model
    Input: Model, test set ie X_test & y_test
    Output: Prints the Classification report
    '''
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print('***')
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))
        results_dict[col] = classification_report(y_test[col], y_pred[:, i], output_dict=True)
    # Convert to df
    weighted_avg = {}
    for key in results_dict.keys():
        weighted_avg[key] = results_dict[key]['weighted avg']

    df_w_avg = pd.DataFrame(weighted_avg).transpose()
    
    print(df_w_avg['f1-score'].mean())
    
    return df_w_avg

In [20]:
perf_report(model, X_test, y_test)

***
related
              precision    recall  f1-score   support

           0       0.74      0.27      0.40      1266
           1       0.80      0.97      0.88      3938
           2       0.80      0.10      0.18        40

    accuracy                           0.79      5244
   macro avg       0.78      0.45      0.48      5244
weighted avg       0.79      0.79      0.76      5244

***
request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      4349
           1       0.91      0.41      0.56       895

    accuracy                           0.89      5244
   macro avg       0.90      0.70      0.75      5244
weighted avg       0.89      0.89      0.87      5244

***
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***
clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5171
           1       0.80      0.05      0.10        73

    accuracy                           0.99      5244
   macro avg       0.89      0.53      0.55      5244
weighted avg       0.98      0.99      0.98      5244

***
money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5140
           1       0.80      0.08      0.14       104

    accuracy                           0.98      5244
   macro avg       0.89      0.54      0.57      5244
weighted avg       0.98      0.98      0.97      5244

***
missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5184
           1       0.00      0.00      0.00        60

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.98      0.9

,precision,recall,f1-score,support
related,0.786214,0.794622,0.755627,5244.0
request,0.893303,0.891876,0.874331,5244.0
offer,0.990108,0.995042,0.992569,5244.0
aid_related,0.774905,0.772693,0.765961,5244.0
medical_help,0.912244,0.922197,0.889024,5244.0
medical_products,0.944459,0.950610,0.929453,5244.0
search_and_rescue,0.974610,0.976926,0.966889,5244.0
security,0.972491,0.983028,0.975171,5244.0
military,0.959351,0.970633,0.957429,5244.0
child_alone,1.000000,1.000000,1.000000,5244.0


In [21]:
df_w_avg['f1-score'].mean()

0.9308950652206912

In [22]:
#overall accuracy
overall_accuracy = (y_pred == y_test).mean().mean()
print('overall accuracy {0:.4f} \n'.format(overall_accuracy))

overall accuracy 0.9454 



### 6. Improve your model
Use grid search to find better parameters. 

In [0]:
parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv = GridSearchCV(model, param_grid=parameters)

In [0]:
cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [0]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [0]:
y_pred = cv.predict(X_test)

In [0]:
#overall accuracy
overall_accuracy = (y_pred == y_test).mean().mean()
print('overall accuracy {0:.4f} \n'.format(overall_accuracy))

overall accuracy 0.9459 



In [0]:
perf_report(cv, X_test, y_test)

***
related
              precision    recall  f1-score   support

           0       0.74      0.27      0.40      1266
           1       0.80      0.97      0.88      3938
           2       0.86      0.15      0.26        40

    accuracy                           0.79      5244
   macro avg       0.80      0.46      0.51      5244
weighted avg       0.78      0.79      0.76      5244

***
request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      4349
           1       0.90      0.41      0.56       895

    accuracy                           0.89      5244
   macro avg       0.90      0.70      0.75      5244
weighted avg       0.89      0.89      0.87      5244

***
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5122
           1       1.00      0.02      0.03       122

    accuracy                           0.98      5244
   macro avg       0.99      0.51      0.51      5244
weighted avg       0.98      0.98      0.97      5244

***
tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5212
           1       0.00      0.00      0.00        32

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      0.99      0.99      5244

***
hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5198
           1       0.00      0.00      0.00        46

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.98      0.99      0.99      5

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [0]:
#Improve  the pipeline
new_model = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [0]:
new_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
                              random_state=None, tol=0.0)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
              

In [0]:
y_pred = new_model.predict(X_test)

In [0]:
#overall accuracy
overall_accuracy = (y_pred == y_test).mean().mean()
print('overall accuracy {0:.4f} \n'.format(overall_accuracy))

overall accuracy 0.9269 



In [0]:
perf_report(new_model, X_test, y_test)

***
related
              precision    recall  f1-score   support

           0       0.50      0.00      0.00      1266
           1       0.75      1.00      0.86      3938
           2       0.71      0.25      0.37        40

    accuracy                           0.75      5244
   macro avg       0.66      0.42      0.41      5244
weighted avg       0.69      0.75      0.65      5244

***
request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      4349
           1       0.00      0.00      0.00       895

    accuracy                           0.83      5244
   macro avg       0.41      0.50      0.45      5244
weighted avg       0.69      0.83      0.75      5244

***
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5122
           1       0.00      0.00      0.00       122

    accuracy                           0.98      5244
   macro avg       0.49      0.50      0.49      5244
weighted avg       0.95      0.98      0.97      5244

***
tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5212
           1       0.00      0.00      0.00        32

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      0.99      0.99      5244

***
hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5198
           1       0.00      0.00      0.00        46

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.98      0.99      0.99      5

In [0]:
 new_parameters = { #'vect__ngram_range': ((1, 1), (1, 2)), 
              #'vect__max_df': (0.5, 1.0), 
              #'vect__max_features': (None, 5000), 
              'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2] }

In [0]:
new_cv = GridSearchCV(new_model, param_grid=new_parameters)
new_cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [0]:
new_cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [0]:
y_pred = new_cv.predict(X_test)

In [0]:
#overall accuracy
overall_accuracy = (y_pred == y_test).mean().mean()
print('overall accuracy {0:.4f} \n'.format(overall_accuracy))

overall accuracy 0.9269 



In [0]:
perf_report(new_cv, X_test, y_test)

***
related
              precision    recall  f1-score   support

           0       0.64      0.01      0.01      1266
           1       0.75      1.00      0.86      3938
           2       0.71      0.25      0.37        40

    accuracy                           0.75      5244
   macro avg       0.70      0.42      0.41      5244
weighted avg       0.72      0.75      0.65      5244

***
request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      4349
           1       0.00      0.00      0.00       895

    accuracy                           0.83      5244
   macro avg       0.41      0.50      0.45      5244
weighted avg       0.69      0.83      0.75      5244

***
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5122
           1       0.00      0.00      0.00       122

    accuracy                           0.98      5244
   macro avg       0.49      0.50      0.49      5244
weighted avg       0.95      0.98      0.97      5244

***
tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5212
           1       0.00      0.00      0.00        32

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      0.99      0.99      5244

***
hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5198
           1       0.00      0.00      0.00        46

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.98      0.99      0.99      5

### 9. Export your model as a pickle file

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/Udacity DS nanodegree/4.Data Enegeneering/Disaster Response Pipeline/model.pkl'
with open(path, 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.